<a href="https://colab.research.google.com/github/SupradeepDanturti/ConvAIProject/blob/main/SpkCounterFromScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install speechbrain

In [ ]:
%%capture
!git clone https://github.com/SupradeepDanturti/ConvAIProject
%cd ConvAIProject
!git config core.sparseCheckout true
!echo '/*' > .git/info/sparse-checkout
!echo '!results/' >> .git/info/sparse-checkout
!echo '!interface/' >> .git/info/sparse-checkout
!git read-tree -mu HEAD

In [ ]:
!python prepare_dataset/download_required_data.py --output_folder ./data

Stage 0: Downloading LibriSpeech
test-clean.tar.gz: 347MB [00:21, 16.1MB/s]               
dev-clean.tar.gz: 338MB [00:19, 17.1MB/s]               
train-clean-100.tar.gz: 6.39GB [05:48, 18.3MB/s]                
Stage 1: Downloading RIRs and Noises
rirs_noises.zip: 1.31GB [01:12, 18.1MB/s]                
Extracting ./data/rirs_noises.zip to ./data


In [ ]:
!cat prepare_dataset/dataset.yaml

# This parameter file is used to create the dataset.


out_folder: ../data # folder where created dataset will be stored.
metadata_folder: !ref <out_folder>/metadata
samplerate: 16000

librispeech_root: !ref <out_folder>/LibriSpeech # e.g., /data/LibriSpeech/ (typically where you've downloaded the data)

librispeech_folders:
  train:
    - !ref <out_folder>/LibriSpeech/train-clean-100/
  dev:
    - !ref <out_folder>/LibriSpeech/dev-clean/
  eval:
    - !ref <out_folder>/LibriSpeech/test-clean/

rirs_noises_root: !ref <out_folder>/RIRS_NOISES
rirs_folders:
  - !ref <out_folder>/RIRS_NOISES/simulated_rirs/
  - !ref <out_folder>/RIRS_NOISES/real_rirs_isotropic_noises
noises_folders:
  - !ref <out_folder>/RIRS_NOISES/pointsource_noises/


# parameters
seed: 1234

n_sessions:
  train: 1000 # This will create 1000 sessions/utterances per class
  dev: 200
  eval: 200
n_speakers: 4 # max number of speakers in each session/utterance created i.e., 0 to n_speakers.

max_length: 120 # max length i

- We'll create only 1 or 2 sessions to make the process faster. Above shown are the original set parameters.

In [ ]:
%%file prepare_dataset/dataset.yaml
# This parameter file is used to create the dataset.
out_folder: ../data # folder where created dataset will be stored.
metadata_folder: !ref <out_folder>/metadata
samplerate: 16000

librispeech_root: !ref <out_folder>/LibriSpeech # e.g., /data/LibriSpeech/
#(typically where you've downloaded the data)

librispeech_folders:
  train:
    - !ref <out_folder>/LibriSpeech/train-clean-100/
  dev:
    - !ref <out_folder>/LibriSpeech/dev-clean/
  eval:
    - !ref <out_folder>/LibriSpeech/test-clean/

rirs_noises_root: !ref <out_folder>/RIRS_NOISES
rirs_folders:
  - !ref <out_folder>/RIRS_NOISES/simulated_rirs/
  - !ref <out_folder>/RIRS_NOISES/real_rirs_isotropic_noises
noises_folders:
  - !ref <out_folder>/RIRS_NOISES/pointsource_noises/


# parameters
seed: 1234

n_sessions:
  train: 2 # This will create 1000 sessions/utterances per class
  dev: 1
  eval: 1
n_speakers: 4 # max number of speakers in each session/utterance created i.e., 0 to n_speakers.

max_length: 120 # max length in seconds for each session/utterance.


Overwriting prepare_dataset/dataset.yaml


In [ ]:
!cat prepare_dataset/create_custom_dataset.py

"""
Script which creates custom dataset based on set parameters[dataset.yaml].

Usage - python create_custom_dataset.py dataset.yaml
"""

import os
import sys
import json
import random
import math
import numpy as np
import speechbrain as sb
from hyperpyyaml import load_hyperpyyaml
from speechbrain.dataio.dataio import read_audio
from speechbrain.utils.data_utils import get_all_files
from local.create_mixtures_metadata import create_metadata
from local.create_mixtures_from_metadata import create_mixture
from pathlib import Path
import torch
import torchaudio
from speechbrain.augment.preparation import write_csv
from speechbrain.augment.time_domain import AddNoise, AddReverb
from tqdm import tqdm

# Load hyperparameters file with command-line overrides
params_file, run_opts, overrides = sb.core.parse_arguments(sys.argv[1:])
with open(params_file) as fin:
    params = load_hyperpyyaml(fin, overrides)

# setting seeds for reproducible code.
np.random.seed(params["seed"])
random.seed(params

- Updating the file paths to make it work for linux based systems
Changed .replace("\ \\") as it is not needed in linux.

In [ ]:
%%file prepare_dataset/create_custom_dataset.py

"""
Script which creates custom dataset based on set parameters[dataset.yaml].

Usage - python create_custom_dataset.py dataset.yaml
"""

import os
import sys
import json
import random
import math
import numpy as np
import speechbrain as sb
from hyperpyyaml import load_hyperpyyaml
from speechbrain.dataio.dataio import read_audio
from speechbrain.utils.data_utils import get_all_files
from local.create_mixtures_metadata import create_metadata
from local.create_mixtures_from_metadata import create_mixture
from pathlib import Path
import torch
import torchaudio
from speechbrain.augment.preparation import write_csv
from speechbrain.augment.time_domain import AddNoise, AddReverb
from tqdm import tqdm

# Load hyperparameters file with command-line overrides
params_file, run_opts, overrides = sb.core.parse_arguments(sys.argv[1:])
with open(params_file) as fin:
    params = load_hyperpyyaml(fin, overrides)

# setting seeds for reproducible code.
np.random.seed(params["seed"])
random.seed(params["seed"])


# we parse the yaml, and create mixtures for every train, dev and eval split.
def split_list(array, split_factors):
    assert round(sum(split_factors), 6) == 1, "split_factors should sum to one"
    np.random.shuffle(array)
    pivots = [int(len(array) * x) for x in split_factors]
    out = []
    indx = 0
    for i in pivots:
        out.append(array[indx: i + indx])
        indx = i
    return out


def parse_libri_folder(libri_folders):
    # parsing librispeech
    utterances = []
    txt_files = []
    for libri_dir in libri_folders:
        utterances.extend(get_all_files(libri_dir, match_and=[".flac"]))
        txt_files.extend(get_all_files(libri_dir, match_and=["trans.txt"]))

    words_dict = {}
    for trans in txt_files:
        with open(trans, "r") as f:
            for line in f:
                splitted = line.split(" ")
                utt_id = splitted[0]
                words = " ".join(splitted[1:])
                words_dict[utt_id] = words.strip("\n")

    speakers = {}
    for u in utterances:
        spk_id = Path(u).parent.parent.stem
        speakers[spk_id] = speakers.get(spk_id, []) + [u]

    return speakers, words_dict


os.makedirs(os.path.join(params["out_folder"], "metadata"), exist_ok=True)

# we generate metadata for each split
for indx, split in enumerate(["train", "dev", "eval"]):
    print("Generating metadata for {} set".format(split))
    # we parse librispeech utterances for current split
    c_libri_folder = params["librispeech_folders"][split]
    c_utterances, c_words = parse_libri_folder(c_libri_folder)

    create_metadata(
        os.path.join(params["out_folder"], "metadata", split),
        params["n_sessions"][split],
        params,
        c_utterances,
        c_words,
    )

# from metadata we generate the actual mixtures
for indx, split in enumerate(["train", "dev", "eval"]):
    print(f"Creating {split} set")
    # load metadata
    metadata_path = os.path.join(params["out_folder"], "metadata", f"{split}.json")
    with open(metadata_path) as f:
        c_meta = json.load(f)

    c_folder = os.path.join(params["out_folder"], split)
    os.makedirs(c_folder, exist_ok=True)

    # Here we loop through sessions with progress tracking
    for sess in tqdm(list(c_meta.keys()), desc=f"Creating {split} set"):
        create_mixture(sess, c_folder, params, c_meta)

print("Creating segments....")


def create_segments(x="train"):
    segment_length = int(params["max_length"] / 60)  # in seconds
    sample_rate = int(params["samplerate"])
    file_list = get_all_files((os.path.join(params["out_folder"], f"{x}")), match_and=[".wav"])

    for file in tqdm(file_list, desc=f"Processing {x} segments"):
        # wav_path = file.replace("\\", "/")  # Windows can be removed if you're using linux
        wav_path = file
        waveform, _ = torchaudio.load(wav_path)
        num_samples_per_segment = sample_rate * segment_length
        total_segments = math.ceil(waveform.size(1) / num_samples_per_segment)

        for segment_id in range(total_segments):
            start_sample = segment_id * num_samples_per_segment
            end_sample = start_sample + num_samples_per_segment

            if end_sample <= waveform.size(1):
                segment_waveform = waveform[:, start_sample:end_sample]

                segment_file_name = f"{os.path.splitext(wav_path)[0]}_{segment_id:03d}_segment.wav"
                torchaudio.save(segment_file_name, segment_waveform, sample_rate)


create_segments("train")
create_segments("dev")
create_segments("eval")
print("Adding Noise....")

"""
Writes all the noises into a csv and creates noisifier and reverber objects
"""
sample_rate = int(params["samplerate"])

rir_audios = get_all_files(os.path.join(params["rirs_noises_root"], "simulated_rirs"), match_and=['.wav'])
rir_audios.extend(
    get_all_files(os.path.join(params["rirs_noises_root"], "real_rirs_isotropic_noises"), match_and=['.wav']))

noise_audios = get_all_files(os.path.join(params["rirs_noises_root"], "pointsource_noises"), match_and=['.wav'])

rir_audios = rir_audios[:5]
noise_audios = noise_audios[:5]
print("Creating simulated rirs csv")
write_csv(rir_audios, os.path.join(params["out_folder"], "simulated_rirs.csv"))
print("Creating noises csv")
write_csv(noise_audios, os.path.join(params["out_folder"], "noises.csv"))

print("Creating noisifier and reverber")
noisifier = AddNoise(os.path.join(params["out_folder"], "noises.csv"), snr_low=5, snr_high=20)
reverber = AddReverb(os.path.join(params["out_folder"], "simulated_rirs.csv"), reverb_sample_rate=sample_rate,
                     clean_sample_rate=sample_rate)

batch_size = 10
probability_noise = 0.5
probability_reverb = 0.5


def load_and_addnoise(x="train"):
    json_data = get_all_files((os.path.join(params["out_folder"], f"{x}")), match_and=[".wav"])
    for data in tqdm(json_data, desc=f'Loading {x}'):
        audio_path = data #.replace("\\", "/")  # Windows can be removed if you're using linux
        signal = read_audio(audio_path)
        clean = signal.unsqueeze(0)

        if random.random() < probability_noise:
            noisy = noisifier(clean, torch.ones(clean.size(0)))
        else:
            noisy = clean

        if noisy.dim() == 2:
            noisy = noisy.unsqueeze(-1)

        if random.random() < probability_reverb:
            reverbed = reverber(noisy)
        else:
            reverbed = noisy

        processed_audio = reverbed.squeeze(0).transpose(0, 1)
        output_path = audio_path

        torchaudio.save(output_path, processed_audio, params["samplerate"])


load_and_addnoise("train")
load_and_addnoise("dev")
load_and_addnoise("eval")
print("AddedNoise")

print("Created Custom Dataset")


Overwriting prepare_dataset/create_custom_dataset.py


In [ ]:
%cd prepare_dataset
!python create_custom_dataset.py dataset.yaml

/content/ConvAIProject/prepare_dataset
Generating metadata for train set
Generating sessions: 100% 2/2 [00:01<00:00,  1.34it/s]
Generating metadata for dev set
Generating sessions: 100% 1/1 [00:00<00:00,  1.44it/s]
Generating metadata for eval set
Generating sessions: 100% 1/1 [00:00<00:00,  1.23it/s]
Creating train set
Creating train set: 100% 10/10 [00:01<00:00,  5.53it/s]
Creating dev set
Creating dev set: 100% 5/5 [00:01<00:00,  3.93it/s]
Creating eval set
Creating eval set: 100% 5/5 [00:01<00:00,  4.51it/s]
Creating segments....
Processing train segments: 100% 10/10 [00:01<00:00,  8.36it/s]
Processing dev segments: 100% 5/5 [00:00<00:00,  8.66it/s]
Processing eval segments: 100% 5/5 [00:00<00:00,  8.20it/s]
Adding Noise....
Creating simulated rirs csv
Creating noises csv
Creating noisifier and reverber
Loading train: 100% 638/638 [00:14<00:00, 42.99it/s]
Loading dev: 100% 323/323 [00:10<00:00, 29.92it/s]
Loading eval: 100% 316/316 [00:09<00:00, 33.66it/s]
AddedNoise
Created Custom

### Train XVector

In [ ]:
%cd /content/ConvAIProject/xvector/

/content/ConvAIProject/xvector


In [ ]:
import speechbrain as sb
import os, sys
from speechbrain.utils.data_utils import get_all_files
from speechbrain.dataio.dataio import read_audio
import json
import torchaudio
from joblib import Parallel, delayed

def process_file(path):
    """
    Processes a single audio file to extract its identifier, path, number of speakers, and length.

    Parameters:
    - path (str): The file path of the audio file.

    Returns:
    - tuple: A tuple containing the audio file identifier and a dictionary with metadata about the audio file.
             The metadata includes the normalized path, number of speakers, and length in seconds.
    """
    parts = path.split("/")[-1].split("_") #.split("\\") can be ignored when using a linux based system
    # parts = path.split("/")[-1].split("\\")[-1].split("_") #.split("\\") can be ignored when using a linux based system
    id = "_".join(parts[:-1])
    num_speakers = parts[3]
    info = torchaudio.info(path)
    length = info.num_frames / 16000

    return id, {
        "wav_path": path, #.replace("\\","/"),  #.split("\\") can be ignored when using a linux based system
        "num_speakers": num_speakers,
        "length": length
    }

def load_json(json_paths, save_file="train"):
    """
    Loads multiple audio files, processes each using `process_file`, and saves the metadata in a JSON file.

    Parameters:
    - json_paths (list): A list of paths to audio files to process.
    - save_file (str): The base name for the output JSON file where the metadata will be saved.

    Returns:
    None. This function generates a JSON file in the '../data/' directory containing the metadata for each audio file.
    """
    data = {}

    # Parallel processing
    results = Parallel(n_jobs=-1, verbose=10)(
        delayed(process_file)(path) for path in json_paths
    )

    for id, path_data in results:
        data[id] = path_data

    with open(f"../data/{save_file}_data.json", 'w') as json_file:
        json.dump(data, json_file, indent=4)

train_files = get_all_files("../data/train", match_and=['_segment.wav'])
test_files = get_all_files("../data/dev", match_and=['_segment.wav'])
valid_files = get_all_files("../data/eval", match_and=['_segment.wav'])

load_json(train_files, save_file="train")
load_json(test_files, save_file="test")
load_json(valid_files, save_file="valid")

test_files_no_spk = [file for file in test_files if 'spk_0' in file]
test_files_1_spk = [file for file in test_files if 'spk_1' in file]
test_files_2_spk = [file for file in test_files if 'spk_2' in file]
test_files_3_spk = [file for file in test_files if 'spk_3' in file]
test_files_4_spk = [file for file in test_files if 'spk_4' in file]

load_json(test_files_no_spk, save_file="test_files_no_spk")
load_json(test_files_1_spk, save_file="test_files_1_spk")
load_json(test_files_2_spk, save_file="test_files_2_spk")
load_json(test_files_3_spk, save_file="test_files_3_spk")
load_json(test_files_4_spk, save_file="test_files_4_spk")

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.19846884186620734s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  22 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.024588584899902344s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.05449652671813965s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done  44 tasks      | elapsed:    2.9s
[Pa

In [ ]:
!python train_xvector_augmentation.py hparams_xvector_augmentation.yaml

speechbrain.core - Beginning experiment!
speechbrain.core - Experiment folder: ../results/XVector/Augmented_2/1986
speechbrain.dataio.encoder - Load called, but CategoricalEncoder is not empty. Loaded data will overwrite everything. This is normal if there is e.g. an unk label defined at init.
speechbrain.core - Info: ckpt_interval_minutes arg from hparam file is used
speechbrain.core - Gradscaler enabled: False. Using precision: fp32.
speechbrain.core - 926.3k trainable parameters in XVectorSpkCounter
speechbrain.utils.checkpoints - Would load a checkpoint here, but none found yet.
speechbrain.utils.epoch_loop - Going into epoch 1
100% 10/10 [00:10<00:00,  1.08s/it, train_loss=0.99]
100% 5/5 [00:01<00:00,  2.79it/s]
speechbrain.nnet.schedulers - Changing lr from 0.001 to 0.001
speechbrain.utils.train_logger - Epoch: 1, lr: 1.00e-03 - train loss: 9.90e-01 - valid loss: 1.56, valid error: 8.07e-01
speechbrain.utils.checkpoints - Saved an end-of-epoch checkpoint in ../results/XVector/Aug

### Train ECAPA-TDNN

In [ ]:
%cd /content/ConvAIProject/ecapa_tdnn/

/content/ConvAIProject/ecapa_tdnn


In [ ]:
!python train_ecapa_tdnn.py hparams_ecapa_tdnn_augmentation.yaml

speechbrain.core - Beginning experiment!
speechbrain.core - Experiment folder: ../results/ECAPA/Augmented/1986
speechbrain.dataio.encoder - Load called, but CategoricalEncoder is not empty. Loaded data will overwrite everything. This is normal if there is e.g. an unk label defined at init.
speechbrain.core - Info: ckpt_interval_minutes arg from hparam file is used
speechbrain.core - Gradscaler enabled: False. Using precision: fp32.
speechbrain.core - 2.0M trainable parameters in ECAPABrain
speechbrain.utils.checkpoints - Would load a checkpoint here, but none found yet.
speechbrain.utils.epoch_loop - Going into epoch 1
100% 10/10 [00:44<00:00,  4.48s/it, train_loss=9.01]
100% 5/5 [00:01<00:00,  2.77it/s]
speechbrain.nnet.schedulers - Changing lr from 3.5e-06 to 3.7e-06
speechbrain.utils.train_logger - epoch: 1, lr: 3.25e-06 - train loss: 9.01 - valid loss: 8.29, valid ErrorRate: 7.91e-01
speechbrain.utils.checkpoints - Saved an end-of-epoch checkpoint in ../results/ECAPA/Augmented/1986

### Train Selfsupervised MLP & XVector

In [ ]:
%cd /content/ConvAIProject/selfsupervised/

/content/ConvAIProject/selfsupervised


In [ ]:
!python selfsupervised_mlp.py hparams_selfsupervised_mlp.yaml

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:363: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
speechbrain.lobes.models.huggingface_transformers.wav2vec2 - wav2vec 2.0 feature extractor is frozen.
speechbrain.core - Beginning experiment!
speechbrain.core - Experiment folder: ../results/train_with_wav2vec2/1993
speechbrai

In [ ]:
!python selfsupervised_xvector.py hparams_selfsupervised_xvector.yaml

config.json: 100% 1.84k/1.84k [00:00<00:00, 9.27MB/s]
/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:363: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
pytorch_model.bin: 100% 380M/380M [00:01<00:00, 254MB/s]
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
preprocessor_config.json: 100% 159/159 [00:00<00:00, 951kB/s]
speechbrain.lobes.models.huggingface_transformers.wav2